## 패키지

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import itertools as it

## 데이터

In [2]:
train = pd.read_csv('./data/train.csv',encoding='cp949')
test = pd.read_csv('./data/test.csv',encoding='cp949')
submission = pd.read_csv('./data/sample_submission.csv',encoding='cp949')
age = pd.read_csv('./data/age_gender_info.csv',encoding='cp949')
car = pd.read_excel('./data/car_age_gender_info.xlsx')

FileNotFoundError: ignored

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
a = train[train['공급유형'] == '공공임대(분납)']
b = train[train['공급유형'] == '공공분양']
c = train[train['공급유형'] == '공공임대(5년)']
d = train[train['공급유형'] == '장기전세']

delete = pd.concat([a,b,c,d],axis=1).index

train2 = train.drop(delete, axis = 0)

NameError: ignored

In [ ]:
train = train2

In [ ]:
train.index

In [ ]:
train.reset_index(inplace=True)

train = train.iloc[:,1:]

In [ ]:
train.columns

In [ ]:
e = test[test['공급유형'] == '공공임대(분납)'].index

In [ ]:
test = test.drop(e, axis = 0)

In [ ]:
test.reset_index(inplace=True)

test = test.iloc[:,1:]

In [ ]:
test.columns

In [ ]:
train.to_csv('train.csv', index=False,encoding='cp949')
test.to_csv('test.csv', index=False,encoding='cp949')

## 인구,차 비율

In [ ]:
# 나이 비율 전처리
# 20대미만 = 10대+20대
# 70대이상 = 70대+80대+90대+100대
age_df = age
age_df.set_index('지역', inplace=True)
age_df.insert(0, '20대미만(여자)', age_df.loc[:,['10대미만(여자)','10대(여자)']].sum(axis=1))
age_df.insert(1, '20대미만(남자)', age_df.loc[:,['10대미만(남자)','10대(남자)']].sum(axis=1))
age_df.insert(24, '70대이상(여자)', age_df.loc[:,['70대(여자)','80대(여자)','90대(여자)','100대(여자)']].sum(axis=1))
age_df.insert(25, '70대이상(남자)', age_df.loc[:,['70대(남자)','80대(남자)','90대(남자)','100대(남자)']].sum(axis=1))
age_df = age_df.drop(['10대미만(여자)','10대(여자)','10대미만(남자)','10대(남자)','70대(여자)','80대(여자)','90대(여자)','100대(여자)','70대(남자)','80대(남자)','90대(남자)','100대(남자)'],axis=1)
print(age_df.shape)
age_df.head(3)

(16, 14)


,20대미만(여자),20대미만(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대이상(여자),70대이상(남자)
지역,,,,,,,,,,,,,,
경상북도,0.086504,0.094555,0.060096,0.067859,0.053433,0.049572,0.083660,0.072613,0.087149,0.072146,0.082684,0.063889,0.082890,0.042950
경상남도,0.080657,0.082470,0.064920,0.070618,0.056414,0.057550,0.077092,0.067600,0.086873,0.072570,0.087201,0.069562,0.080807,0.045667
대전광역시,0.068687,0.071884,0.060834,0.064247,0.068654,0.066848,0.074667,0.067925,0.085751,0.068819,0.088468,0.070261,0.090257,0.052700


In [ ]:
# 자동차 비율 전처리
car_df = pd.DataFrame(np.random.rand(16,14),
                      columns = age_df.columns,
                      index = age_df.index)
car_df.at[:,:] = np.NaN
print(car_df.shape)
car_df.head(3)

(16, 14)


,20대미만(여자),20대미만(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대이상(여자),70대이상(남자)
지역,,,,,,,,,,,,,,
경상북도,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
경상남도,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대전광역시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 나이 데이터에 없는 인천은 삭제
city_notnull = car.index[car['행정구역별'].notnull()].tolist()
city_notnull.remove(0)
city_notnull.remove(85)
city_notnull

[1, 29, 57, 113, 141, 169, 197, 225, 253, 281, 309, 337, 365, 393, 421]

In [ ]:
# 나이, 성별, 지역별 비율 계산
# 해당 비율 =  자동차대수 / 남여 총자동차대수
for j in city_notnull:
    total = car['2010'][j]+car['2010'][j+14]
    rate_woman = [round((car['2010'][j+i*2+14]+car['2010'][j+i*2+15])/total,4) if i!=0 else round(car['2010'][j+15]/total,4) for i in range(0,7)]
    rate_man = [round((car['2010'][j+i*2]+car['2010'][j+i*2+1])/total,4) if i!=0 else round(car['2010'][j+1]/total,4) for i in range(0,7)]
    rate_list = list(it.chain(*zip(rate_woman, rate_man)))
    car_df.loc[car['행정구역별'][j]] = rate_list
print(car_df.shape)
car_df.head(3)

(16, 14)


,20대미만(여자),20대미만(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대이상(여자),70대이상(남자)
지역,,,,,,,,,,,,,,
경상북도,0.0001,0.0004,0.0132,0.0424,0.0258,0.1834,0.0352,0.2538,0.0303,0.2242,0.0145,0.1204,0.0106,0.0457
경상남도,0.0001,0.0001,0.0097,0.0367,0.0279,0.2151,0.0396,0.2811,0.0316,0.2115,0.0129,0.0957,0.0093,0.0288
대전광역시,0.0002,0.0003,0.0141,0.0386,0.0398,0.2113,0.0478,0.2724,0.0382,0.2035,0.0132,0.0886,0.0058,0.0260


In [ ]:
# 나이비율 * 자동차비율
age_car_df = pd.DataFrame(age_df.values * car_df.values, columns=age_df.columns, index=age_df.index)
print(age_car_df.shape)
age_car_df.head(3)

(16, 14)


,20대미만(여자),20대미만(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),40대(남자),50대(여자),50대(남자),60대(여자),60대(남자),70대이상(여자),70대이상(남자)
지역,,,,,,,,,,,,,,
경상북도,0.000009,0.000038,0.000793,0.002877,0.001379,0.009092,0.002945,0.018429,0.002641,0.016175,0.001199,0.007692,0.000879,0.001963
경상남도,0.000008,0.000008,0.000630,0.002592,0.001574,0.012379,0.003053,0.019002,0.002745,0.015348,0.001125,0.006657,0.000752,0.001315
대전광역시,0.000014,0.000022,0.000858,0.002480,0.002732,0.014125,0.003569,0.018503,0.003276,0.014005,0.001168,0.006225,0.000523,0.001370


In [ ]:
# 나이*자동차 비율을 전체합 1이 되게 다시 계산
rate_age_car = age_car_df.sum(axis=1)
rate_age_car[-1] = rate_age_car[2]
rate_age_car = [round(i/sum(rate_age_car),4) for i in rate_age_car]
rate_age_car_df = pd.DataFrame(rate_age_car, columns=['rate'], index=age_df.index)
rate_age_car_df

,rate
지역,
경상북도,0.0612
경상남도,0.0622
대전광역시,0.0638
경기도,0.0649
전라북도,0.0604
강원도,0.0620
광주광역시,0.0648
충청남도,0.0631
부산광역시,0.0614


In [ ]:
rate_age_car_df_train = [rate_age_car_df.loc[i][0] for i in train['지역']]
rate_age_car_df_test = [rate_age_car_df.loc[i][0] for i in test['지역']]
train.insert(14, '인구차비율', rate_age_car_df_train)
test.insert(14, '인구차비율', rate_age_car_df_test)

In [ ]:
print(train.shape)
train.head(3)

(2921, 16)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,인구차비율,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38,A,15667000,103680,0.0,3.0,1425,0.0612,1015
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38,A,15667000,103680,0.0,3.0,1425,0.0612,1015
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38,A,27304000,184330,0.0,3.0,1425,0.0612,1015


In [ ]:
print(test.shape)
test.head(3)

(1016, 15)


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,인구차비율
0,C1072,754,아파트,경기도,국민임대,39.79,116,14,H,22830000,189840,0.0,2,683,0.0649
1,C1072,754,아파트,경기도,국민임대,46.81,30,14,A,36048000,249930,0.0,2,683,0.0649
2,C1072,754,아파트,경기도,국민임대,46.90,112,14,H,36048000,249930,0.0,2,683,0.0649


In [ ]:
train = train.replace('-', np.NaN)
test = test.replace('-', np.NaN)

In [ ]:
train = train.astype({'임대보증금':'float','임대료':'float'})
test = test.astype({'임대보증금':'float','임대료':'float'})

## 결측치 확인

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2921 non-null   object 
 1   총세대수                          2921 non-null   int64  
 2   임대건물구분                        2921 non-null   object 
 3   지역                            2921 non-null   object 
 4   공급유형                          2921 non-null   object 
 5   전용면적                          2921 non-null   float64
 6   전용면적별세대수                      2921 non-null   int64  
 7   공가수                           2921 non-null   int64  
 8   자격유형                          2921 non-null   object 
 9   임대보증금                         2347 non-null   float64
 10  임대료                           2347 non-null   float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2720 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2917 non-null   float64
 13  단지내

In [ ]:
train.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           574
임대료                             574
도보 10분거리 내 지하철역 수(환승노선 수 반영)    201
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
인구차비율                             0
등록차량수                             0
dtype: int64

In [ ]:
test.isna().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           186
임대료                             186
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
인구차비율                             0
dtype: int64

## 결측치 처리

In [ ]:
# -값 NuN로 변경
train = train.replace('-', np.NaN)
test = test.replace('-', np.NaN)

In [ ]:
# 지하철, 버스 결측값 0
fillname = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
for i in fillname:
    train[i] = train[i].fillna(0)
    test[i] = test[i].fillna(0)

In [ ]:
# 자격유형 결측값 A
test['자격유형'] = test['자격유형'].fillna('A')

In [ ]:
# 임대보증금, 임대료 type: float
train = train.astype({'임대보증금':'float','임대료':'float'})
test = test.astype({'임대보증금':'float','임대료':'float'})

In [ ]:
# 지역별 임대보증금, 임대료 평균값
security_deposit_train = dict()   # 임대보증금
rent_train = dict()               # 임대료
security_deposit_test = dict()   # 임대보증금
rent_test = dict()               # 임대료

for i in set(train['지역']):
    security_deposit_train[i] = np.mean(train[train['지역'].isin([i])].dropna(axis=0)['임대보증금'])
    rent_train[i] = np.mean(train[train['지역'].isin([i])].dropna(axis=0)['임대료'])

for i in set(test['지역']):
    security_deposit_test[i] = np.mean(test[test['지역'].isin([i])].dropna(axis=0)['임대보증금'])
    rent_test[i] = np.mean(test[test['지역'].isin([i])].dropna(axis=0)['임대료'])

In [ ]:
security_deposit_train

{'경상북도': 19863428.57142857,
 '광주광역시': 28588851.063829787,
 '충청남도': 19085356.097560976,
 '강원도': 18005500.0,
 '전라북도': 19154669.724770643,
 '충청북도': 22785411.042944785,
 '대구광역시': 26995212.12121212,
 '서울특별시': 58842695.652173914,
 '경기도': 32726694.041867957,
 '세종특별자치시': 46912765.957446806,
 '제주특별자치도': 17339196.96969697,
 '경상남도': 21923533.088235293,
 '부산광역시': 23786718.446601942,
 '대전광역시': 22692864.285714287,
 '전라남도': 15928927.152317882,
 '울산광역시': 27965250.0}

In [ ]:
rent_train

{'경상북도': 147809.32773109243,
 '광주광역시': 163677.7304964539,
 '충청남도': 120378.29268292683,
 '강원도': 183339.27631578947,
 '전라북도': 134853.3027522936,
 '충청북도': 163466.87116564417,
 '대구광역시': 263120.10101010103,
 '서울특별시': 448403.26086956525,
 '경기도': 226381.884057971,
 '세종특별자치시': 492170.21276595746,
 '제주특별자치도': 158130.45454545456,
 '경상남도': 165003.05147058822,
 '부산광역시': 200043.49514563108,
 '대전광역시': 169308.2142857143,
 '전라남도': 132250.0,
 '울산광역시': 219009.16666666666}

In [ ]:
# 임대보증금, 임대료 NaN값 0으로 채우기
train = train.fillna(0)
test = test.fillna(0)

In [ ]:
# 임대보증금, 임대료 0값을 지역별 평균값으로 채우기
for i in range(len(train)):
    if train['임대보증금'][i] == 0:
        train['임대보증금'][i] = security_deposit_train[train['지역'][i]]
    if train['임대료'][i] == 0:
        train['임대료'][i] = rent_train[train['지역'][i]]

for i in range(len(test)):
    if test['임대보증금'][i] == 0:
        test['임대보증금'][i] = security_deposit_test[test['지역'][i]]
    if test['임대료'][i] == 0:
        test['임대료'][i] = rent_test[test['지역'][i]]

<ipython-input-742-1007d25a33eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['임대보증금'][i] = security_deposit_train[train['지역'][i]]
<ipython-input-742-1007d25a33eb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['임대료'][i] = rent_train[train['지역'][i]]
<ipython-input-742-1007d25a33eb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['임대보증금'][i] = security_deposit_test[test['지역'][i]]
<ipython-input-742-1007d25a33eb>

In [ ]:
train.head(162)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,인구차비율,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38,A,1.566700e+07,103680.000000,0.0,3.0,1425,0.0612,1015
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38,A,1.566700e+07,103680.000000,0.0,3.0,1425,0.0612,1015
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38,A,2.730400e+07,184330.000000,0.0,3.0,1425,0.0612,1015
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38,A,2.730400e+07,184330.000000,0.0,3.0,1425,0.0612,1015
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38,A,2.730400e+07,184330.000000,0.0,3.0,1425,0.0612,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,C2621,1396,아파트,대전광역시,영구임대,31.32,298,26,C,6.873000e+06,94990.000000,1.0,4.0,277,0.0638,133
158,C2621,1396,아파트,대전광역시,영구임대,52.74,9,26,C,1.157400e+07,159960.000000,1.0,4.0,277,0.0638,133
159,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26,C,1.157400e+07,159960.000000,1.0,4.0,277,0.0638,133
160,C2621,1396,아파트,대전광역시,영구임대,52.74,4,26,C,1.157400e+07,159960.000000,1.0,4.0,277,0.0638,133


In [ ]:
train.isna().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           0
임대료                             0
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
인구차비율                           0
등록차량수                           0
dtype: int64

In [ ]:
test.isna().sum()

단지코드                            0
총세대수                            0
임대건물구분                          0
지역                              0
공급유형                            0
전용면적                            0
전용면적별세대수                        0
공가수                             0
자격유형                            0
임대보증금                           0
임대료                             0
도보 10분거리 내 지하철역 수(환승노선 수 반영)    0
도보 10분거리 내 버스정류장 수              0
단지내주차면수                         0
인구차비율                           0
dtype: int64

## 컬럼명 변경

지하철역 수와 버스 정류장 수의 컬럼명을 지하철, 버스로 변경하였습니다.

In [ ]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '인구차비율', '등록차량수'],
      dtype='object')

In [ ]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '인구차비율', '등록차량수']

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '자격유형',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '인구차비율']

# 원핫인코딩

In [ ]:
from sklearn.preprocessing import OneHotEncoder

np.array(train.iloc[:,2]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(train.iloc[:,2]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(train.iloc[:,2]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'임대건물구분':list(train.iloc[:,2])})
species_building12=pd.get_dummies(test_df)

np.array(train.iloc[:,3]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(train.iloc[:,3]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(train.iloc[:,3]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'지역':list(train.iloc[:,3])})
area12=pd.get_dummies(test_df)

np.array(train.iloc[:,4]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(train.iloc[:,4]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(train.iloc[:,4]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'공급유형':list(train.iloc[:,4])})
supply_type12=pd.get_dummies(test_df)

np.array(train.iloc[:,8]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(train.iloc[:,8]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(train.iloc[:,8]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'자격유형':list(train.iloc[:,8])})
st12=pd.get_dummies(test_df)

np.array(test.iloc[:,2]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(test.iloc[:,2]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(test.iloc[:,2]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'임대건물구분':list(test.iloc[:,2])})
species_building1=pd.get_dummies(test_df)

np.array(test.iloc[:,3]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(test.iloc[:,3]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(test.iloc[:,3]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'지역':list(test.iloc[:,3])})
area1=pd.get_dummies(test_df)

np.array(test.iloc[:,4]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(test.iloc[:,4]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(test.iloc[:,4]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'공급유형':list(test.iloc[:,4])})
supply_type1=pd.get_dummies(test_df)

np.array(test.iloc[:,8]).reshape(-1,1)
oneHot_encoder = OneHotEncoder()
oneHot_encoder.fit(np.array(test.iloc[:,8]).reshape(-1,1))
oneHot_labels = oneHot_encoder.transform(np.array(test.iloc[:,8]).reshape(-1,1))
oneHot_labels.toarray()
oneHot_labels.shape
test_df = pd.DataFrame({'자격유형':list(test.iloc[:,8])})
st1=pd.get_dummies(test_df)

In [ ]:
area1.insert(8,'지역_서울특별시',0)

In [ ]:
st1.insert(1,'자격유형_B',0)
st1.insert(5,'자격유형_F',0)
st1.insert(14,'자격유형_O',0)

## 총 면적 ( sum(면적x세대수) - 면적평균x공가수 )

In [ ]:
train_total_area = []
for i in train['단지코드']:
    area = train[train['단지코드'].isin([i])]['전용면적'].tolist()
    num_house = train[train['단지코드'].isin([i])]['전용면적별세대수'].tolist()
    vac_house = train[train['단지코드'].isin([i])]['공가수'].tolist()
    train_total_area.append(int(sum([area[j]*num_house[j] for j in range(len(area))])-np.mean(area)*vac_house[0]))

test_total_area = []
for i in test['단지코드']:
    area = test[test['단지코드'].isin([i])]['전용면적'].tolist()
    num_house = test[test['단지코드'].isin([i])]['전용면적별세대수'].tolist()
    vac_house = test[test['단지코드'].isin([i])]['공가수'].tolist()
    test_total_area.append(int(sum([area[j]*num_house[j] for j in range(len(area))])-np.mean(area)*vac_house[0]))

In [ ]:
train.insert(8, '총면적', train_total_area,True)
test.insert(8, '총면적', test_total_area,True)

In [ ]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,총면적,자격유형,임대보증금,임대료,지하철,버스,단지내주차면수,인구차비율,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38,43710,A,15667000.0,103680.0,0.0,3.0,1425,0.0612,1015
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38,43710,A,15667000.0,103680.0,0.0,3.0,1425,0.0612,1015
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38,43710,A,27304000.0,184330.0,0.0,3.0,1425,0.0612,1015
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38,43710,A,27304000.0,184330.0,0.0,3.0,1425,0.0612,1015
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38,43710,A,27304000.0,184330.0,0.0,3.0,1425,0.0612,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,C2532,239,아파트,강원도,국민임대,49.20,19,7,11926,A,11346000.0,116090.0,0.0,1.0,166,0.0620,146
2917,C2532,239,아파트,강원도,국민임대,51.08,34,7,11926,A,14005000.0,142310.0,0.0,1.0,166,0.0620,146
2918,C2532,239,아파트,강원도,국민임대,51.73,34,7,11926,A,14005000.0,142310.0,0.0,1.0,166,0.0620,146
2919,C2532,239,아파트,강원도,국민임대,51.96,114,7,11926,A,14005000.0,142310.0,0.0,1.0,166,0.0620,146


## 총 임대보증금, 총 임대료
#### ( sum(임대보증금x세대수) - 임대보증금평균x공가수 )
#### ( sum(임대료x세대수) - 임대료x공가수 )

In [ ]:
train_total_security_deposit = []
train_totla_rent = []
for i in train['단지코드']:
    sec = train[train['단지코드'].isin([i])]['임대보증금'].tolist()
    ren = train[train['단지코드'].isin([i])]['임대료'].tolist()
    num_house = train[train['단지코드'].isin([i])]['전용면적별세대수'].tolist()
    vac_house = train[train['단지코드'].isin([i])]['공가수'].tolist()
    train_total_security_deposit.append(int(sum([sec[j]*num_house[j] for j in range(len(sec))])-np.mean(sec)*vac_house[0]))
    train_totla_rent.append(int(sum([ren[j]*num_house[j] for j in range(len(ren))])-np.mean(ren)*vac_house[0]))

test_total_security_deposit = []
test_totla_rent = []
for i in test['단지코드']:
    sec = test[test['단지코드'].isin([i])]['임대보증금'].tolist()
    ren = test[test['단지코드'].isin([i])]['임대료'].tolist()
    num_house = test[test['단지코드'].isin([i])]['전용면적별세대수'].tolist()
    vac_house = test[test['단지코드'].isin([i])]['공가수'].tolist()
    test_total_security_deposit.append(int(sum([sec[j]*num_house[j] for j in range(len(sec))])-np.mean(sec)*vac_house[0]))
    test_totla_rent.append(int(sum([ren[j]*num_house[j] for j in range(len(ren))])-np.mean(ren)*vac_house[0]))

In [ ]:
train.insert(13, '총임대보증금', train_total_security_deposit,True)
train.insert(14, '총임대료', train_totla_rent,True)
test.insert(13, '총임대보증금', test_total_security_deposit,True)
test.insert(14, '총임대료', test_totla_rent,True)

In [ ]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,총면적,자격유형,임대보증금,임대료,지하철,총임대보증금,총임대료,버스,단지내주차면수,인구차비율,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38,43710,A,15667000.0,103680.0,0.0,22160742750,150074410,3.0,1425,0.0612,1015
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38,43710,A,15667000.0,103680.0,0.0,22160742750,150074410,3.0,1425,0.0612,1015
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38,43710,A,27304000.0,184330.0,0.0,22160742750,150074410,3.0,1425,0.0612,1015
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38,43710,A,27304000.0,184330.0,0.0,22160742750,150074410,3.0,1425,0.0612,1015
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38,43710,A,27304000.0,184330.0,0.0,22160742750,150074410,3.0,1425,0.0612,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,C2532,239,아파트,강원도,국민임대,49.20,19,7,11926,A,11346000.0,116090.0,0.0,3169034833,32236246,1.0,166,0.0620,146
2917,C2532,239,아파트,강원도,국민임대,51.08,34,7,11926,A,14005000.0,142310.0,0.0,3169034833,32236246,1.0,166,0.0620,146
2918,C2532,239,아파트,강원도,국민임대,51.73,34,7,11926,A,14005000.0,142310.0,0.0,3169034833,32236246,1.0,166,0.0620,146
2919,C2532,239,아파트,강원도,국민임대,51.96,114,7,11926,A,14005000.0,142310.0,0.0,3169034833,32236246,1.0,166,0.0620,146


## 전용면적을 5의 배수로 변경

In [ ]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

## 전용면적 상/하한 적용

상한100, 하한 15

In [ ]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [ ]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

In [ ]:
print(train.shape)
print(test.shape)

(2921, 19)
(1016, 18)


In [ ]:
train2=train.drop(['등록차량수'],axis=1)

In [ ]:
train2=pd.concat([train2,species_building12,area12,supply_type12,st12],axis=1) # 한 번만
test2=pd.concat([test,species_building1,area1,supply_type1,st1],axis=1) # 한 번만

In [ ]:
train2=train2.drop(['임대건물구분','지역','공급유형'],axis=1)
test2=test2.drop(['임대건물구분','지역','공급유형'],axis=1)

print(train2.shape)
print(test2.shape)

(2921, 54)
(1016, 54)


In [ ]:
train2.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '자격유형', '임대보증금',
       '임대료', '지하철', '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가',
       '임대건물구분_아파트', '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시',
       '지역_대구광역시', '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시',
       '지역_울산광역시', '지역_전라남도', '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_국민임대', '공급유형_영구임대',
       '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D',
       '자격유형_E', '자격유형_F', '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K',
       '자격유형_L', '자격유형_M', '자격유형_N', '자격유형_O'],
      dtype='object')

In [ ]:
test2.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '자격유형', '임대보증금',
       '임대료', '지하철', '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가',
       '임대건물구분_아파트', '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시',
       '지역_대구광역시', '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시',
       '지역_울산광역시', '지역_전라남도', '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_국민임대', '공급유형_영구임대',
       '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D',
       '자격유형_E', '자격유형_F', '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K',
       '자격유형_L', '자격유형_M', '자격유형_N', '자격유형_O'],
      dtype='object')

In [ ]:
train=pd.concat([train2,train.iloc[:,18]],axis=1)

In [ ]:
test = test2

In [ ]:
train.shape

(2921, 55)

In [ ]:
test.shape

(1016, 54)

## 단지별 데이터 1차원으로 취합

In [ ]:
columns = ['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '자격유형', '임대보증금',
       '임대료', '지하철', '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가',
       '임대건물구분_아파트', '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시',
       '지역_대구광역시', '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시',
       '지역_울산광역시', '지역_전라남도', '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_국민임대', '공급유형_영구임대',
       '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D',
       '자격유형_E', '자격유형_F', '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K',
       '자격유형_L', '자격유형_M', '자격유형_N', '자격유형_O']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [ ]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

421it [00:07, 56.90it/s]
150it [00:02, 56.44it/s]


In [ ]:
new_train.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '자격유형', '임대보증금',
       '임대료', '지하철', '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가',
       '임대건물구분_아파트', '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시',
       '지역_대구광역시', '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시',
       '지역_울산광역시', '지역_전라남도', '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_국민임대', '공급유형_영구임대',
       '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D',
       '자격유형_E', '자격유형_F', '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K',
       '자격유형_L', '자격유형_M', '자격유형_N', '자격유형_O', '면적_35.0', '면적_50.0', '면적_55.0',
       '면적_30.0', '면적_45.0', '면적_40.0', '면적_25.0', '면적_70.0', '면적_15.0',
       '면적_20.0', '면적_100.0', '면적_60.0', '면적_75.0', '면적_80.0', '면적_65.0',
       '등록차량수'],
      dtype='object')

In [ ]:
new_test.columns

Index(['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '자격유형', '임대보증금',
       '임대료', '지하철', '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가',
       '임대건물구분_아파트', '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시',
       '지역_대구광역시', '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시',
       '지역_울산광역시', '지역_전라남도', '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_국민임대', '공급유형_영구임대',
       '공급유형_임대상가', '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D',
       '자격유형_E', '자격유형_F', '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K',
       '자격유형_L', '자격유형_M', '자격유형_N', '자격유형_O', '면적_35.0', '면적_50.0', '면적_55.0',
       '면적_30.0', '면적_45.0', '면적_40.0', '면적_25.0', '면적_70.0', '면적_15.0',
       '면적_20.0', '면적_100.0', '면적_60.0', '면적_75.0', '면적_80.0', '면적_65.0'],
      dtype='object')

## 상관계수

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
corr=new_train.corr(method='pearson')
corr

,총세대수,전용면적,전용면적별세대수,공가수,총면적,임대보증금,임대료,지하철,총임대보증금,총임대료,...,면적_25.0,면적_70.0,면적_15.0,면적_20.0,면적_100.0,면적_60.0,면적_75.0,면적_80.0,면적_65.0,등록차량수
총세대수,1.000000,-0.125226,0.476171,0.297151,0.847749,0.037030,0.042432,0.083932,0.650011,0.646151,...,0.577496,0.053825,0.048290,0.253369,0.321147,0.048796,0.012962,0.005620,0.038264,0.590944
전용면적,-0.125226,1.000000,-0.123989,-0.208605,0.260720,0.565580,0.709547,-0.029436,0.091236,0.312965,...,-0.370001,0.453072,-0.314355,-0.224829,-0.180457,-0.066313,0.187087,0.554796,0.160037,0.328923
전용면적별세대수,0.476171,-0.123989,1.000000,0.046826,0.295194,-0.149262,-0.100265,0.127284,0.056793,0.125900,...,0.582909,-0.072564,-0.050258,0.056257,0.561998,0.128611,-0.049536,-0.064487,-0.039418,-0.009198
공가수,0.297151,-0.208605,0.046826,1.000000,0.228157,0.049166,-0.115801,-0.102418,0.332286,0.125223,...,0.031539,-0.008223,0.305266,0.236889,-0.134688,-0.051824,0.005557,-0.024516,-0.022066,0.174311
총면적,0.847749,0.260720,0.295194,0.228157,1.000000,0.249604,0.303349,0.063006,0.734932,0.842286,...,0.231729,0.332157,-0.092619,0.085252,0.089685,-0.020213,0.042178,0.296922,0.143762,0.747009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
면적_60.0,0.048796,-0.066313,0.128611,-0.051824,-0.020213,-0.092235,-0.056318,0.030408,-0.065125,-0.038738,...,0.163422,-0.016387,-0.014258,-0.016615,0.325313,1.000000,-0.006049,-0.017915,-0.005700,-0.100411
면적_75.0,0.012962,0.187087,-0.049536,0.005557,0.042178,0.116267,0.237516,-0.020576,0.036210,0.097222,...,-0.016257,0.094927,-0.013003,-0.014304,-0.003854,-0.006049,1.000000,0.193837,0.009862,0.088495
면적_80.0,0.005620,0.554796,-0.064487,-0.024516,0.296922,0.476461,0.614902,-0.051758,0.257241,0.425106,...,-0.083387,0.604575,-0.038640,-0.042450,-0.047712,-0.017915,0.193837,1.000000,0.032585,0.230941
면적_65.0,0.038264,0.160037,-0.039418,-0.022066,0.143762,0.093169,0.208231,0.027315,0.087616,0.223041,...,-0.026541,0.404538,-0.012295,-0.013507,-0.015213,-0.005700,0.009862,0.032585,1.000000,0.187048


## 데이터 추출

In [ ]:
new_train2 = new_train.loc[:,['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '임대보증금', '임대료', '지하철',
       '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가', '임대건물구분_아파트',
       '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시', '지역_대구광역시',
       '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시', '지역_울산광역시', '지역_전라남도',
       '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)',
       '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택',
       '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F', '자격유형_G',
       '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M', '자격유형_N',
       '자격유형_O','등록차량수']]
new_train2

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,총면적,임대보증금,임대료,지하철,총임대보증금,...,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O,등록차량수
0,C2483,900.0,35.0,134.0,38.0,43710.0,15667000.0,103680.0,0.0,2.216074e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,C2515,545.0,30.0,276.0,17.0,21187.0,9216000.0,82940.0,0.0,7.364220e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
2,C1407,1216.0,30.0,288.0,13.0,48622.0,15620000.0,127350.0,1.0,3.319702e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1064.0
3,C1945,755.0,35.0,120.0,6.0,37659.0,15607000.0,130070.0,1.0,1.681320e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.0
4,C1470,696.0,35.0,228.0,14.0,30649.0,12015000.0,102130.0,0.0,1.325010e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,C2586,90.0,15.0,12.0,7.0,2452.0,16800000.0,71400.0,0.0,2.406480e+09,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,57.0
417,C2035,492.0,25.0,132.0,24.0,18005.0,10000000.0,117000.0,0.0,8.577480e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,246.0
418,C2020,40.0,15.0,20.0,7.0,831.0,18000000.0,87000.0,1.0,8.790667e+08,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,19.0
419,C2437,90.0,20.0,90.0,12.0,1936.0,10346000.0,107530.0,0.0,8.069880e+08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0


In [ ]:
new_test2 = new_test.loc[:,['단지코드', '총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '임대보증금', '임대료', '지하철',
       '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율', '임대건물구분_상가', '임대건물구분_아파트',
       '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시', '지역_대구광역시',
       '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시', '지역_울산광역시', '지역_전라남도',
       '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)',
       '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택',
       '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F', '자격유형_G',
       '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M', '자격유형_N',
       '자격유형_O']]
new_test2

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,총면적,임대보증금,임대료,지하철,총임대보증금,...,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O
0,C1072,754.0,35.0,116.0,14.0,35716.0,2.283000e+07,189840.000000,0.0,2.791404e+10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1128,1354.0,35.0,368.0,9.0,63014.0,2.283000e+07,189840.000000,0.0,4.725583e+10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C1456,619.0,30.0,82.0,18.0,27148.0,1.970600e+07,156200.000000,0.0,2.057372e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C1840,593.0,35.0,253.0,7.0,25467.0,1.441800e+07,108130.000000,0.0,1.058040e+10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,C1332,1297.0,35.0,282.0,11.0,58945.0,2.859800e+07,203050.000000,0.0,5.155739e+10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,C2456,349.0,25.0,24.0,17.0,12071.0,6.992000e+06,117000.000000,0.0,4.503361e+09,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,C1266,596.0,25.0,164.0,35.0,19965.0,8.084000e+06,149910.000000,0.0,8.316554e+09,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,C2152,120.0,20.0,66.0,9.0,3202.0,1.559958e+07,161812.542373,0.0,1.731553e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,C1267,675.0,20.0,28.0,38.0,20526.0,6.882000e+06,104370.000000,0.0,1.304021e+10,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Min-Max 정규화

In [ ]:
train_code = new_train2.loc[:,['단지코드']]
y_train = new_train2.iloc[:,-1]
test_code = new_test2.loc[:,['단지코드']]

In [ ]:
#정규화하지 않을 칼럼
train2 = new_train2.iloc[:,14:-1]
test2 = new_test2.iloc[:,14:]

In [ ]:
# 정규화할 칼럼
new_train2 = new_train2.iloc[:,1:14]
new_test2 = new_test2.iloc[:,1:14]

In [ ]:
# MinMaxScaler 정규화

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(new_train2)
scaler.fit(new_test2)

# 정규화 적용 후 새로운 변수명으로 저장

new_train3 = scaler.transform(new_train2)
new_test3 = scaler.transform(new_test2)

In [ ]:
# 분리했던 데이터와 합치기 위해 데이터 프레임으로 변경

new_train3 = pd.DataFrame(new_train3)
new_test3 = pd.DataFrame(new_test3)

In [ ]:
# 데이터 합치기

new_train4 = pd.concat([train2,new_train3], axis = 1)
new_test4 = pd.concat([test2,new_test3], axis = 1)

In [ ]:
# 정규화 후 칼럼명 재설정
# 정규화하면 칼럼명도 변경됨

new_train4.columns=['임대건물구분_상가', '임대건물구분_아파트',
       '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시', '지역_대구광역시',
       '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시', '지역_울산광역시', '지역_전라남도',
       '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)',
       '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택',
       '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F', '자격유형_G',
       '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M', '자격유형_N',
       '자격유형_O','총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '임대보증금', '임대료', '지하철',
       '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율']
new_test4.columns=['임대건물구분_상가', '임대건물구분_아파트',
       '지역_강원도', '지역_경기도', '지역_경상남도', '지역_경상북도', '지역_광주광역시', '지역_대구광역시',
       '지역_대전광역시', '지역_부산광역시', '지역_서울특별시', '지역_세종특별자치시', '지역_울산광역시', '지역_전라남도',
       '지역_전라북도', '지역_제주특별자치도', '지역_충청남도', '지역_충청북도',
       '공급유형_공공임대(10년)', '공급유형_공공임대(50년)',
       '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_행복주택',
       '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F', '자격유형_G',
       '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M', '자격유형_N',
       '자격유형_O','총세대수', '전용면적', '전용면적별세대수', '공가수', '총면적', '임대보증금', '임대료', '지하철',
       '총임대보증금', '총임대료', '버스', '단지내주차면수', '인구차비율']

## 학습

In [ ]:
y_train = pd.DataFrame(y_train)

In [ ]:
x_train = new_train4
x_test = new_test4

In [ ]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
model.fit(x_train, y_train)

<ipython-input-852-139fa7af76d2>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

## 추론 및 제출

In [ ]:
pred = model.predict(x_test)

In [ ]:
submission['num'] = pred

In [ ]:
submission['num']

0       744.02
1      1339.21
2       545.32
3       517.80
4      1259.78
        ...   
145     222.81
146     462.41
147      68.51
148     388.56
149     251.64
Name: num, Length: 150, dtype: float64

In [ ]:
submission.to_csv('공급유형삭제_원핫인코딩_민맥스(맹기버전).csv', index=False)